<a href="https://colab.research.google.com/github/fitzroypet/Background-remove-U-2-Net-Demo/blob/main/Copy_of_Copy_of_U_2_Netp_Demonstration_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# U-2-NETp DEMONSTRATION

**Using U-2-NETp for :**
 - **Background Removal**
 - **Bounding Box Creation**
 - **Salient Feature Highlighting**


 U-2-NET Paper: [U2-Net: Going Deeper with Nested U-Structure for Salient Object Detection](https://arxiv.org/abs/2005.09007)

 Original Repo: [U-2-Net Github repo](https://github.com/vaibhavmit074/U-2-Net-Demo)

References: X. Qin, Z. Zhang, C. Huang, M. Dehghan, O. R. Zaiane, and M. Jagersand, “U2-net: Going deeper with nested u-structure for salient object
detection,” Pattern Recognition, vol. 106, p. 107404, 2020

# SETUP

- Clone github repo: [https://github.com/vaibhavmit074/U-2-Net-Demo](https://github.com/vaibhavmit074/U-2-Net-Demo)
- Make directories for images
- Import necessary packages

Note: Uses U-2-Netp with 4.7MB weights, not U-2-Net with 176.3MB

# Make sure runtype is GPU
**Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU**

In [1]:
#@title Run this cell for setups{display-mode: "form"}
%cd /content
!/usr/local/cuda/bin/nvcc --version
!git clone https://github.com/shreyas-bk/U-2-Net
%cd /content/U-2-Net
print('making images directory')
!mkdir images
print('making results directory')
!mkdir results
print('importing...')
from google.colab import files
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
from PIL import Image as Img
import cv2
from google.colab.patches import cv2_imshow
print('Done!')

/content
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Cloning into 'U-2-Net'...
remote: Enumerating objects: 520, done.
remote: Total 520 (delta 0), reused 0 (delta 0), pack-reused 520
Receiving objects: 100% (520/520), 12.39 MiB | 16.32 MiB/s, done.
Resolving deltas: 100% (236/236), done.
/content/U-2-Net
making images directory
making results directory
importing...
Done!


# UPLOAD IMAGES

In [2]:
#@title Run this cell to upload images{display-mode: "form"}
%cd /content/U-2-Net/images
uploaded = files.upload()
%cd /content/U-2-Net

/content/U-2-Net/images


Saving image (2).JPG to image (2).JPG
Saving image (3).JPG to image (3).JPG
Saving image (4).JPG to image (4).JPG
Saving image (5).JPG to image (5).JPG
Saving image (6).JPG to image (6).JPG
Saving image (7).JPG to image (7).JPG
Saving image (8).JPG to image (8).JPG
Saving image (9).JPG to image (9).JPG
Saving image (10).JPG to image (10).JPG
Saving image (11).JPG to image (11).JPG
Saving image (12).JPG to image (12).JPG
Saving image (13).JPG to image (13).JPG
Saving image (14).JPG to image (14).JPG
Saving image (15).JPG to image (15).JPG
Saving image (16).JPG to image (16).JPG
Saving image (17).JPG to image (17).JPG
Saving image (18).JPG to image (18).JPG
Saving image (19).JPG to image (19).JPG
Saving image (20).JPG to image (20).JPG
Saving image (21).JPG to image (21).JPG
Saving image (22).JPG to image (22).JPG
Saving image (23).JPG to image (23).JPG
Saving image (24).JPG to image (24).JPG
Saving image (25).JPG to image (25).JPG
Saving image (26).JPG to image (26).JPG
Saving image (27

# RESULTS
 - Run u2net_test.py
 - Remove Background
 - Create Bounding Box
 - Highlight Salient Features

In [3]:
#@title Run this cell for results{display-mode: "form"}

# This code will be hidden when the notebook is loaded.

!python -W ignore u2net_test.py

image_dir = os.path.join(os.getcwd(), 'images')
names = [name[:-4] for name in os.listdir(image_dir)]
THRESHOLD = 0.9
RESCALE = 255
LAYER = 2
COLOR = (0, 0, 0)
THICKNESS = 4
SAL_SHIFT = 100


for name in names:

  # BACKGROUND REMOVAL

  output = load_img('/content/U-2-Net/results/'+name+'.png')
  out_img = img_to_array(output)
  out_img /= RESCALE

  out_img[out_img > THRESHOLD] = 1
  out_img[out_img <= THRESHOLD] = 0

  shape = out_img.shape
  a_layer_init = np.ones(shape = (shape[0],shape[1],1))
  mul_layer = np.expand_dims(out_img[:,:,0],axis=2)
  a_layer = mul_layer*a_layer_init
  rgba_out = np.append(out_img,a_layer,axis=2)

  input = load_img('/content/U-2-Net/images/'+name+'.jpg')
  inp_img = img_to_array(input)
  inp_img /= RESCALE

  a_layer = np.ones(shape = (shape[0],shape[1],1))
  rgba_inp = np.append(inp_img,a_layer,axis=2)

  rem_back = (rgba_inp*rgba_out)
  rem_back_scaled = rem_back*RESCALE

  # BOUNDING BOX CREATION

  out_layer = out_img[:,:,LAYER]
  x_starts = [np.where(out_layer[i]==1)[0][0] if len(np.where(out_layer[i]==1)[0])!=0 else out_layer.shape[0]+1 for i in range(out_layer.shape[0])]
  x_ends = [np.where(out_layer[i]==1)[0][-1] if len(np.where(out_layer[i]==1)[0])!=0 else 0 for i in range(out_layer.shape[0])]
  y_starts = [np.where(out_layer.T[i]==1)[0][0] if len(np.where(out_layer.T[i]==1)[0])!=0 else out_layer.T.shape[0]+1 for i in range(out_layer.T.shape[0])]
  y_ends = [np.where(out_layer.T[i]==1)[0][-1] if len(np.where(out_layer.T[i]==1)[0])!=0 else 0 for i in range(out_layer.T.shape[0])]
  
  startx = min(x_starts)
  endx = max(x_ends)
  starty = min(y_starts)
  endy = max(y_ends)
  start = (startx,starty)
  end = (endx,endy)

  box_img = inp_img.copy()
  box_img = cv2.rectangle(box_img, start, end, COLOR, THICKNESS)
  box_img = np.append(box_img,a_layer,axis=2)
  box_img_scaled = box_img*RESCALE

  # SALIENT FEATURE MAP

  sal_img = inp_img.copy()
  add_layer = out_img.copy()
  add_layer[add_layer==1] = SAL_SHIFT/RESCALE
  sal_img[:,:,LAYER] += add_layer[:,:,LAYER]
  sal_img = np.append(sal_img,a_layer,axis=2)
  sal_img_scaled = sal_img*RESCALE
  sal_img_scaled[sal_img_scaled>RESCALE] = RESCALE

  # OUTPUT RESULTS

  inp_img*=RESCALE
  inp_img = np.append(inp_img,RESCALE*a_layer,axis=2)
  inp_img = cv2.resize(inp_img,(int(shape[1]/3),int(shape[0]/3)))
  rem_back = cv2.resize(rem_back_scaled,(int(shape[1]/3),int(shape[0]/3)))
  box_img = cv2.resize(box_img_scaled,(int(shape[1]/3),int(shape[0]/3)))
  sal_img = cv2.resize(sal_img_scaled,(int(shape[1]/3),int(shape[0]/3)))
  result = np.concatenate((inp_img,rem_back,box_img,sal_img),axis=1)
  result_img = Img.fromarray(result.astype('uint8'), 'RGBA')
  print('\nINPUT                                    BACKGROUND REMOVED                     BOUNDING BOX                               SALIENT MAP\n')
  display(result_img)

['/content/U-2-Net/images/image (657).JPG', '/content/U-2-Net/images/image (847).JPG', '/content/U-2-Net/images/image (1473).JPG', '/content/U-2-Net/images/image (373).JPG', '/content/U-2-Net/images/image (1791).JPG', '/content/U-2-Net/images/image (2119).JPG', '/content/U-2-Net/images/image (1120).JPG', '/content/U-2-Net/images/image (1191).JPG', '/content/U-2-Net/images/image (938).JPG', '/content/U-2-Net/images/image (2081).JPG', '/content/U-2-Net/images/image (1357).JPG', '/content/U-2-Net/images/image (1143).JPG', '/content/U-2-Net/images/image (446).JPG', '/content/U-2-Net/images/image (1858).JPG', '/content/U-2-Net/images/image (138).JPG', '/content/U-2-Net/images/image (43).JPG', '/content/U-2-Net/images/image (1302).JPG', '/content/U-2-Net/images/image (1524).JPG', '/content/U-2-Net/images/image (1805).JPG', '/content/U-2-Net/images/image (1184).JPG', '/content/U-2-Net/images/image (1457).JPG', '/content/U-2-Net/images/image (685).JPG', '/content/U-2-Net/images/image (1140).JP

FileNotFoundError: ignored